In [4]:
%pip install pymupdf pypdf langchain_community langchain langchain_google_genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install -U langchain langchain-core langchain-google-genai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
%pip install -U langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import re
from typing import List
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader
# ✅ 수정: 모듈 경로 변경 (langchain.text_splitter -> langchain_text_splitters)
from langchain_text_splitters import RecursiveCharacterTextSplitter 
import os

# --- 1. 뉴욕시 문서 전처리 함수 ---
def clean_documents_nyc(docs: List[Document]) -> List[Document]:
    """영문 보고서 패턴(Chapter, Page numbers) 노이즈를 제거합니다."""
    cleaned_docs = []
    
    # 영문 문서의 일반적인 헤더/푸터 패턴: Chapter, Section 제목, 페이지 번호 등
    nyc_header_pattern = re.compile(
        # 'CHAPTER X', 'SECTION 3.2', 'Figure 1-2' 등 제거
        r'(CHAPTER|SECTION|FIGURE|TABLE)\s+[:.\s\w\d-]+', 
        re.IGNORECASE | re.DOTALL
    )
    # 페이지 번호와 관련된 흔한 패턴: 'PAGE X of Y', '- X -', 숫자만 있는 푸터 등
    nyc_page_pattern = re.compile(r'PAGE\s+\d+\s+of\s+\d+|p\.\s*\d+|-\s*\d+\s*-|\n\d+\n', re.IGNORECASE)
    
    for doc in docs:
        cleaned_content = doc.page_content
        
        # 1. 영문 헤더/섹션 패턴 제거
        cleaned_content = re.sub(nyc_header_pattern, '', cleaned_content)
        # 2. 영문 페이지 번호 패턴 제거
        cleaned_content = re.sub(nyc_page_pattern, '', cleaned_content)
        
        # 3. 과도한 공백 및 개행 문자 정리 (공통)
        cleaned_content = re.sub(r'\s{2,}', ' ', cleaned_content)
        cleaned_content = re.sub(r'\n\s*\n', '\n\n', cleaned_content)
        
        # 50자 미만인 노이즈 청크는 버립니다.
        if len(cleaned_content.strip()) > 50:
            cleaned_docs.append(Document(page_content=cleaned_content.strip(), metadata=doc.metadata))
            
    return cleaned_docs

# --- 2. 문서 로딩 및 전처리 적용 ---
pdf_path_nyc = r"C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap9\OneNYC_2050_Strategic_Plan.pdf"

# 2.1. PDF 로드 (사용자 코드)
loader = PyPDFLoader(pdf_path_nyc)
data_nyc_raw = loader.load() # 변수명을 raw로 변경하여 클리닝 전 데이터임을 명확히 합니다.
for d in data_nyc_raw:
    d.metadata["city"] = "뉴욕시"  # ⭐ 도시 태그 부여

print(f"로드된 뉴욕시 문서 페이지 수: {len(data_nyc_raw)} 페이지")

# 2.2. 텍스트 노이즈 클리닝 적용 (추가 단계) ⬅️ ⭐ 노이즈 제거
cleaned_docs_nyc = clean_documents_nyc(data_nyc_raw) 
print(f"클리닝 후 유효 문서 청크 수 (페이지별): {len(cleaned_docs_nyc)}개")

로드된 뉴욕시 문서 페이지 수: 332 페이지
클리닝 후 유효 문서 청크 수 (페이지별): 327개


In [8]:
# --- 3. 텍스트 분할 (Splitter) 적용 ---

# 3.1. RecursiveCharacterTextSplitter 초기화 (사용자 요청: chunk_size=500, chunk_overlap=100)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=100,
    separators=["\n\n\n", "\n\n", "\n", " ", ""] # 영문에도 적합한 구분자
)

# 3.2. 클리닝된 문서(cleaned_docs_nyc)를 분할
all_splits = text_splitter.split_documents(cleaned_docs_nyc)

# 4. 결과 출력
print(f"\n✅ 최종 생성된 청크 개수 (500자 단위): {len(all_splits)}개")
print("\n--- 첫 3개 청크 내용 출력 (클리닝 확인용) ---")
for i, d in enumerate(all_splits[:3]):
    print(f"[청크 {i+1} - 길이 {len(d.page_content)}]")
    print(d.page_content)
    print("-----")


✅ 최종 생성된 청크 개수 (500자 단위): 2378개

--- 첫 3개 청크 내용 출력 (클리닝 확인용) ---
[청크 1 - 길이 215]
OneNYC BUILDING A STRONG AND FAIR CITY VOLUME 1 OF 9 A
PRIL 2019
THE CITY OF NEW YORK
MAYOR BILL DE BLASIO
DEAN FULEIHAN FIRST DEPUTY MAYOR D
OMINIC WILLIAMS CHIEF POLICY ADVISOR
D
ANIEL A. ZARRILLI One NYC DIRECTOR
-----
[청크 2 - 길이 357]
2 | O neNYC 2050 NYC.GOV/O neNYC
ONENYC 2050 IS A STRATEGY TO SECURE OUR CITY’S FUTURE AGAINST THE CHALLENGES OF TODAY AND TOMORROW. WITH BOLD ACTIONS TO CONFRONT OUR CLIMATE CRISIS, ACHIEVE EQUITY, AND STRENGTHEN OUR DEMOCRACY, WE ARE BUILDING A STRONG AND FAIR CITY. JOIN US.
OneNYC 2050
BUILDING A STRONG AND FAIR CITY
MODERN
INFRASTRUCTURE
VOLUME 9 OF 9
-----
[청크 3 - 길이 498]
OneNYC 2050
BUILDING A STRONG AND FAIR CITY
MODERN
INFRASTRUCTURE
VOLUME 9 OF 9
New York City will invest in reliable physical and digital infrastructure that is readyto meet the needs of a 21st century city. New York City will grow and diversify its economy so that it creates opportunity for all, s

In [9]:
for i, split in enumerate(all_splits):
    print(f"Split {i+1}:-----------------------------------------\n")
    print(split)

Split 1:-----------------------------------------

page_content='OneNYC BUILDING A STRONG AND FAIR CITY VOLUME 1 OF 9 A
PRIL 2019
THE CITY OF NEW YORK
MAYOR BILL DE BLASIO
DEAN FULEIHAN FIRST DEPUTY MAYOR D
OMINIC WILLIAMS CHIEF POLICY ADVISOR
D
ANIEL A. ZARRILLI One NYC DIRECTOR' metadata={'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign 14.0 (Windows)', 'creationdate': '2019-04-30T13:48:30-04:00', 'moddate': '2020-01-03T15:55:12-05:00', 'title': '', 'trapped': '/False', 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap9\\OneNYC_2050_Strategic_Plan.pdf', 'total_pages': 332, 'page': 0, 'page_label': '1', 'city': '뉴욕시'}
Split 2:-----------------------------------------

page_content='2 | O neNYC 2050 NYC.GOV/O neNYC
ONENYC 2050 IS A STRATEGY TO SECURE OUR CITY’S FUTURE AGAINST THE CHALLENGES OF TODAY AND TOMORROW. WITH BOLD ACTIONS TO CONFRONT OUR CLIMATE CRISIS, ACHIEVE EQUITY, AND STRENGTHEN OUR DEMOCRACY, WE ARE BUILDING A STRONG A

In [10]:
print(type(all_splits[0]))

<class 'langchain_core.documents.base.Document'>


In [11]:
import re
from typing import List
from langchain_core.documents import Document
# 🚨 수정된 부분: langchain.text_splitter 대신 langchain_text_splitters 사용
from langchain_text_splitters import RecursiveCharacterTextSplitter 
from langchain_community.document_loaders import PyPDFLoader
import os

# --- 텍스트 클리닝 함수 (서울시용) ---
# 이전 답변에서 clean_documents로 정의되었으나, 여기서는 clean_text_noise 함수 내에서 전체 처리를 가정합니다.
def clean_and_split_documents(docs: List[Document]) -> List[Document]:
    cleaned_docs = []
    # 검색 결과로 확인된 불필요한 패턴들 (페이지 헤더/푸터, 장 제목 등)
    header_pattern = re.compile(r'\d+\s*제\d+장\s*[\w\s·]+|\d+\s*제\d+절\s*[\w\s·]+|\n\d{1,3}제\d장[\w\s]*', re.DOTALL)
    
    for doc in docs:
        cleaned_content = doc.page_content
        
        # 1. 페이지 번호, 장/절 제목 등 반복되는 헤더 패턴 제거
        cleaned_content = re.sub(header_pattern, '', cleaned_content)
        
        # 2. 과도한 공백 및 개행 문자 정리
        cleaned_content = re.sub(r'\s{2,}', ' ', cleaned_content)  
        cleaned_content = re.sub(r'\n\s*\n', '\n\n', cleaned_content)
        
        # 내용이 충분히 남아있다면 Document를 다시 생성
        if len(cleaned_content.strip()) > 50: 
            cleaned_docs.append(Document(page_content=cleaned_content.strip(), metadata=doc.metadata))
            
    return cleaned_docs

# --- 문서 로딩 및 전처리 적용 ---
# pdf_path는 사용 환경에 맞게 조정 필요
pdf_path = r"C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap9\2040_seoul_plan.pdf"
loader_seoul = PyPDFLoader(pdf_path)
raw_docs = loader_seoul.load()
for d in raw_docs:
    d.metadata["city"] = "서울시"  # ⭐ 도시 태그 부여
    
# 2. 텍스트 노이즈 제거 (전처리 적용)
cleaned_docs = clean_and_split_documents(raw_docs) # 함수명을 일관성 있게 변경했습니다.

# 3. 텍스트 분할 (chunk_size: 1000, overlap: 200 등)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=200,
    separators=["\n\n\n", "\n\n", "\n", " ", ""]
)
seoul_splits = text_splitter.split_documents(cleaned_docs)

print(f"✅ 전처리 후 생성된 유효 청크 개수: {len(all_splits)}개")

# 4. ChromaDB 재생성/업데이트는 all_splits를 사용하여 다음 단계에서 실행
# ...

✅ 전처리 후 생성된 유효 청크 개수: 2378개


In [12]:
for i, split in enumerate(seoul_splits):
    print(f"Split {i+1}:---------------------------------\n")
    print(split)

Split 1:---------------------------------

page_content='「2040 서울도시기본계획」을 발간하며
지난 3년간 코로나19 팬데믹으로 전 세계가 심각한 타격을 받아왔지만, 대한민국의 수도 서울은 혁신적인 디지털 기술과 뛰어난 시민 의식, 풍부한 자연환경을 토대로 도시의 가능성과 잠재력을 확인할 수 있었습니다. 「2040 서울도시기본계획」은 기후위기, 디지털 전환, 생활양식의 변화 등 글로벌 대도시가 당면한 과제에 대한 해법을 제시하고 있습니다.첫째, 보행일상권으로의 공간구조 개편입니다. 서울을 하나의 기준으로 관리하던 지금까지의 방식에서 벗어나, 미래의 서울은 다양한 지역특성을 반영한 차별화된 계획체계를 통해 동네단위의 자족적 생활권으로 재구성하게 됩니다. 이는 기후위기, 팬데믹 등 각종 재난상황에서도 도시활동이 가능한 공간단위로, 서울 어디에서나 시민 삶의 질이 보장되는 새로운 차원의 균형발전정책 기반이 될 것입니다.둘째, 과감하고 유연한 도시계획 기조로의 전환입니다. 앞으로의 도시계획은 미래 여건변화에 유연하고 신속하게 대응할 수 있는 체계를 통해 현장에서 강력하게 작동하는 수단이 될 것입니다. 미래지향적인 계획철학을 토대로 융복합적인 토지이용제도를 실현하고 수변녹지와 연계된 생활공간을 조성하는 등 도시계획체계를 과감하고 유연하게 전환했습니다.' metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2020 11.0.0.5178', 'creationdate': '2024-12-12T18:16:11+09:00', 'author': 'SI', 'moddate': '2024-12-12T18:16:11+09:00', 'pdfversion': '1.4', 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap9\\2040_seoul_plan.pdf', 'total_pag

In [13]:
print(seoul_splits[50].page_content)
print('---------------------------')
print(seoul_splits[51].page_content)

제1절 서울의 변화진단33-노인여가복지시설 역시 확충되고 있으나, 노령화의 속도를 따라잡지 못해 2020년에는 노인 천 명당 2.1개소로 2010년에 비해 0.9개소 감소Ÿ다양한 생활서비스 시설의 양적 공급은 긍정적이지만, 지역 수요에 알맞은 생활서비스 시설을 도입하여 질적으로 서비스를 개선할 수 있도록 해야 한다.
[그림 2-15] 노인여가복지시설 및 재가노인복지시설 현황(2015~2020)자료: 서울시 인생이모작지원과, 노인여가복지시설 통계, 각 연도; 어르신복지과, 재가노인복지시설 통계, 각 연도 세계 대도시와 비교해서 부족한 생활 녹지인프라Ÿ2014년 기준 서울의 1인당 공원면적은 16.2㎡로 런던의 33.4㎡에는 미치지 못하지만, 싱가포르, 베이징, 뉴욕 등과 유사한 면적이며, 파리보다 큰 편이다.3)-서울은 2020년 기준 16.87㎡로 1인당 공원면적이 다소 증가. 단, 북한산 및 도시자연공원 등 산지 면적을 제외하면 1인당 공원면적은 5.71㎡로 감소Ÿ디지털 전환과 팬데믹을 거치면서 여가공간에 대한 시민수요가 급증하고 있으며 특히, 생활권 내 공원녹지와 오픈스페이스의 양·질적인 수요변화에 적극적 대응이 필요하다. [그림 2-16] 세계 주요 도시의 1인당 공원면적(2014)자료: 서울연구데이터서비스(공원녹지 부문)3) 서울연구원 서울연구데이터서비스 생활인프라 공원녹지 부문 참조해 재가공
---------------------------
. 서울의 미래 여건 변화와 과제1) 가속화되는 저출생·고령화2040년 서울의 고령인구는 약 32%, 늘어나는 복지·의료 수요 대응 필요Ÿ장기적인 저출생·고령화로 인해 인구변화 속도는 둔화되는 반면, 기술발전에 따른 평균수명 증가로 고령인구의 비율은 지속적으로 상승 중이다. Ÿ서울의 고령화 속도는 빠른 편으로 노년인구 비율은 2018년 기준 14.4%로 고령사회에 진입하였으며, 2026년에는 초고령화사회에 진입할 것으로 예측된다.2040년 서울도시기본계획 계획인구는 통계청 전망을 반영한 854만 명으로 설정Ÿ204

In [14]:
for i in range(len(seoul_splits)-1):
    seoul_splits[i].page_content +="\n"+seoul_splits[i+1].page_content[:100]

print(seoul_splits[50].page_content)
print('---------------------------')
print(seoul_splits[51].page_content)

제1절 서울의 변화진단33-노인여가복지시설 역시 확충되고 있으나, 노령화의 속도를 따라잡지 못해 2020년에는 노인 천 명당 2.1개소로 2010년에 비해 0.9개소 감소Ÿ다양한 생활서비스 시설의 양적 공급은 긍정적이지만, 지역 수요에 알맞은 생활서비스 시설을 도입하여 질적으로 서비스를 개선할 수 있도록 해야 한다.
[그림 2-15] 노인여가복지시설 및 재가노인복지시설 현황(2015~2020)자료: 서울시 인생이모작지원과, 노인여가복지시설 통계, 각 연도; 어르신복지과, 재가노인복지시설 통계, 각 연도 세계 대도시와 비교해서 부족한 생활 녹지인프라Ÿ2014년 기준 서울의 1인당 공원면적은 16.2㎡로 런던의 33.4㎡에는 미치지 못하지만, 싱가포르, 베이징, 뉴욕 등과 유사한 면적이며, 파리보다 큰 편이다.3)-서울은 2020년 기준 16.87㎡로 1인당 공원면적이 다소 증가. 단, 북한산 및 도시자연공원 등 산지 면적을 제외하면 1인당 공원면적은 5.71㎡로 감소Ÿ디지털 전환과 팬데믹을 거치면서 여가공간에 대한 시민수요가 급증하고 있으며 특히, 생활권 내 공원녹지와 오픈스페이스의 양·질적인 수요변화에 적극적 대응이 필요하다. [그림 2-16] 세계 주요 도시의 1인당 공원면적(2014)자료: 서울연구데이터서비스(공원녹지 부문)3) 서울연구원 서울연구데이터서비스 생활인프라 공원녹지 부문 참조해 재가공
. 서울의 미래 여건 변화와 과제1) 가속화되는 저출생·고령화2040년 서울의 고령인구는 약 32%, 늘어나는 복지·의료 수요 대응 필요Ÿ장기적인 저출생·고령화로 인해 인구변화 속
---------------------------
. 서울의 미래 여건 변화와 과제1) 가속화되는 저출생·고령화2040년 서울의 고령인구는 약 32%, 늘어나는 복지·의료 수요 대응 필요Ÿ장기적인 저출생·고령화로 인해 인구변화 속도는 둔화되는 반면, 기술발전에 따른 평균수명 증가로 고령인구의 비율은 지속적으로 상승 중이다. Ÿ서울의 고령화 속도는 빠른 편으로 노년인구 비율은 2018년 

In [15]:
print(len(all_splits))
all_splits.extend(seoul_splits)
print(len(all_splits))

2378
2646


In [16]:
%pip install langchain_chroma

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings # Gemini 임베딩 모델
from dotenv import load_dotenv
import os

# --- 환경 설정 ---
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

if not GOOGLE_API_KEY:
    raise ValueError("환경 변수 'GOOGLE_API_KEY'를 설정해주세요. (.env 파일 확인)")

# ✅ 수정: 임베딩 전용 모델을 지정합니다.
# 'text-embedding-004'는 Google의 RAG/검색 작업에 널리 사용되는 안정적인 임베딩 모델입니다.
# 참고: 모델명은 API 제공사의 정책에 따라 변경될 수 있습니다.
EMBEDDING_MODEL_NAME = "text-embedding-004" 

# Gemini Embedding 모델 설정
embedding_model = GoogleGenerativeAIEmbeddings(
    model=EMBEDDING_MODEL_NAME, 
    api_key=GOOGLE_API_KEY 
)

# --- 임베딩 실행 (원본 코드 1:1 대응) ---
query_text = "서울시의 주거 정책을 알려줘"

print(f"🚀 질의: \"{query_text}\"")
print(f"⚙️ 모델: {EMBEDDING_MODEL_NAME} 사용")

# query_text를 벡터로 변환합니다.
v = embedding_model.embed_query(query_text)

# 결과 출력
print("\n--- Gemini 임베딩 결과 ---")
print(f"✅ 생성된 벡터 (앞 5개 요소): {v[:5]}...")
print(f"✅ 벡터의 차원(Dimension): {len(v)}")

🚀 질의: "서울시의 주거 정책을 알려줘"
⚙️ 모델: text-embedding-004 사용

--- Gemini 임베딩 결과 ---
✅ 생성된 벡터 (앞 5개 요소): [-0.004858795087784529, 0.021195709705352783, -0.014273482374846935, -0.030111584812402725, 0.02831087075173855]...
✅ 벡터의 차원(Dimension): 768


In [18]:
from langchain_chroma import Chroma
import os
persist_directory= r'..\chroma_store'
if not os.path.exists(persist_directory):
    print("Creating new Chroma store")
    vectorstore = Chroma.from_documents(
        documents=all_splits,
        embedding=embedding_model,
        persist_directory= persist_directory
    )
else:
    print("Loading existing Chroma store")
    vectorstore= Chroma(
        persist_directory= persist_directory,
        embedding_function= embedding_model
    )

def detect_city_from_query(query: str) -> str:
    q = query.lower()

    ny_keywords = ["뉴욕", "new york", "nyc"]
    seoul_keywords = ["서울", "seoul"]

    if any(k in q for k in ny_keywords):
        return "뉴욕시"
    if any(k in q for k in seoul_keywords):
        return "서울시"

    return "서울시"  # default


def city_retriever(city: str):
    return vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={
            'k': 10,
            'fetch_k': 50,
            'lambda_mult': 0.8,
            'filter': {'city': {'$eq': city}},   # ⭐ 핵심
        }
    )

Creating new Chroma store


In [19]:
# ✅ 질의
query = "서울시의 주거 정책을 알려줘"

# 도시 감지
city = detect_city_from_query(query)

# 도시별 리트리버
retriever = city_retriever(city)

# ✅ ✅ 기존: get_relevant_documents → ❌
# docs = retriever.get_relevant_documents(query)

# ✅ ✅ 변경: invoke() 사용
docs = retriever.invoke(query)

# 출력
for d in docs:
    print(d)
    print("------")


page_content='[표 3-4] 경제·산업 부문별 전략계획
-1 서울의 도시경쟁력 강화를 위한 서울형 미래 신산업 육성2-1-1 미래 신기술 산업 육성을 통한 성장 동력 지속적 확보Ÿ심화되는 글로벌 경쟁 속에서 미래 성장을 선도할 수 있도' metadata={'page_label': '81', 'page': 80, 'pdfversion': '1.4', 'city': '서울시', 'producer': 'Hancom PDF 1.3.0.542', 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap9\\2040_seoul_plan.pdf', 'author': 'SI', 'moddate': '2024-12-12T18:16:11+09:00', 'creator': 'Hwp 2020 11.0.0.5178', 'total_pages': 205, 'creationdate': '2024-12-12T18:16:11+09:00'}
------
page_content='지역중심 중 신설동역 일대를 공공용지와 청계천 주변을 활용하여 관광·업무·상업·문화 중심으로 육성한다.Ÿ망우 지역중심의 경우 우수한 교통 여건을 기반으로 동북부 수도권의 업무·상업·문화 중심지로 역할을 할 수 있도록 중심기능 및 인접지역과의 연계성을 강화한다.-경기 북부(의정부, 동두천 등)와 경기 동북부(구리, 남양주 등) 지역과 교통연계 강화Ÿ미아 지역중심의 경우 미아사거리역 일대를 중심으로 자치구의 여건을 반영하여 문화·쇼핑·업무가 유기적으로 연계된 중심지로 육성한다.
.Ÿ광운대 역세권 일대를 신설·물류 부지를 활용하여 동북권의 새로운 활력을 불어넣는 신 경제거점으로 육성하고 새로운 일자리를 창출시켜 동북권의 자족성을 강화한다.Ÿ창동차량기지, 도' metadata={'total_pages': 205, 'creator': 'Hwp 2020 11.0.0.5178', 'moddate': '2024-12-12T1

In [20]:
# --- 1. 환경 설정 및 LLM 초기화 ---
from typing import List
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import os

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if not GOOGLE_API_KEY:
    raise ValueError("환경 변수 'GOOGLE_API_KEY'를 설정해주세요. (.env 파일 확인)")

chat = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GOOGLE_API_KEY,
    temperature=0.0
)

# --- 2. 프롬프트 (컨텍스트 없으면 명확히 선언 + 도시 일탈 금지 규칙) ---
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "아래 문맥(Context)에 근거해 한국어로만 간결하게 답하라. "
     "문맥에 없으면 '제공된 문서에는 해당 정보가 없습니다.'라고 답하라.\n"
     "반드시 {city} 관련 내용만 답하고, 다른 도시는 언급하지 마라.\n\n"
     "Context:\n{context}"),
    MessagesPlaceholder("messages"),
    ("human", "{query}")
])

# --- 3. 문서 결합(stuffing) ---
def stuff(docs: List[Document]) -> str:
    if not docs:
        return ""  # 컨텍스트 없음 → 프롬프트 규칙에 따라 '없음'으로 답하게 됨
    return "\n\n".join(d.page_content for d in docs)

# --- 4. 도시 감지 → 도시별 리트리브 → 컨텍스트/도시 주입 ---
def retrieve_with_city(inputs: dict) -> dict:
    q = inputs.get("query", "")
    # ⚠️ detect_city_from_query / city_retriever 는 앞서 정의한 것을 그대로 사용
    city = detect_city_from_query(q)
    retr = city_retriever(city)  # 필터가 포함된 retriever
    docs = retr.invoke(q)        # VectorStoreRetriever는 invoke()가 표준
    return {
        "city": city,
        "context": docs,         # 그 다음 단계에서 stuff로 문자열화
        "messages": inputs.get("messages", []),
        "query": q
    }

# --- 5. LCEL 체인 구성 ---
document_chain = (
    RunnableLambda(retrieve_with_city) |
    RunnableMap({
        "city":    RunnableLambda(lambda x: x["city"]),
        "context": RunnableLambda(lambda x: stuff(x["context"])),
        "messages": RunnableLambda(lambda x: x.get("messages", [])),
        "query":   RunnableLambda(lambda x: x["query"]),
    }) |
    prompt |
    chat
)

print(f"✅ Gemini LLM ({chat.model}) 기반 문서 체인(도시 필터 RAG) 설정 완료.")


✅ Gemini LLM (models/gemini-2.5-flash) 기반 문서 체인(도시 필터 RAG) 설정 완료.


In [21]:
from langchain_core.messages import HumanMessage, AIMessage

user_question = "서울시의 주거 정책을 알려줘"

# 1) 도시 감지 → 도시별 리트리버 생성 (필수!)
city = detect_city_from_query(user_question)         # "서울시" 또는 "뉴욕시"
retriever = city_retriever(city)                     # filter={'city': {'$eq': city}}

# 2) 검색 (VectorStoreRetriever는 invoke 사용)
docs = retriever.invoke(user_question)               # List[Document]

# 3) 대화 이력
chat_history = [HumanMessage(content=user_question)]

# 4) LLM 호출
#   - 프롬프트가 {context}와 MessagesPlaceholder("messages")만 요구한다면:
# answer = document_chain.invoke({"messages": chat_history, "context": docs})

#   - 프롬프트가 {city}도 요구한다면(제가 제안한 프롬프트처럼):
answer = document_chain.invoke({
    "messages": chat_history,
    "context": docs,
    "city": city,
    "query": user_question,   # 프롬프트에 {query}가 있다면 함께 전달
})

# 5) 이력 업데이트 & 출력
chat_history.append(AIMessage(content=answer.content))
print(answer.content)


서울시의 주거 정책은 다음과 같습니다:
*   양질의 주택 공급 확대를 위해 도시계획체계를 유연화하고 효율적인 정비사업을 유도합니다.
*   저이용·유휴지 활용 등 다양한 도시 계획적 수단을 통해 양질의 주택 공급을 활성화합니다.
*   주거·업무·여가 등 다양한 생활을 누릴 수 있는 다기능 복합지역 조성을 지향합니다.


In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

query_for_nyc = "뉴욕은?"

# ✅ 질의 확장 프롬프트 (간결 + 강제 규칙)
query_augmentation_prompt = ChatPromptTemplate.from_messages(
    [
        # 1) 규칙을 system으로 최우선 배치
        (
            "system",
            (
                "역할: 기존 대화의 주제를 유지하되, {query}가 지칭한 도시를 중심으로 "
                "한 문장의 명료한 질문으로 재작성한다.\n"
                "규칙:\n"
                "  - 반드시 결과 문장에 정확히 '뉴욕시'를 포함한다.\n"
                "  - 기존 대화에서 다루던 주제(예: 주거 정책/주택 정책/주거 안정 등)는 유지한다.\n"
                "  - 대명사(이/그/저/거기 등)는 구체적 명사로 치환한다.\n"
                "  - 최종 출력은 한국어 한 문장만.\n"
            ),
        ),
        # 2) 이후에 과거 대화 컨텍스트를 배치
        MessagesPlaceholder(variable_name="messages"),
        # 3) 사용자 현재 질의는 별도 human 메시지로
        ("human", "{query}"),
    ]
)

query_augmentation_chain = query_augmentation_prompt | chat | StrOutputParser()

# ⚠️ chat_history는 messages 리스트 형태여야 합니다.
# 예) chat_history.messages 가 있다면 그걸 넣으세요.
augmented_query = query_augmentation_chain.invoke({
    "messages": chat_history,
    "query": query_for_nyc
})

print(augmented_query)


뉴욕시의 주거 정책은 무엇인가요?


In [23]:
# ✅ 1) 도시 자동 감지
city = detect_city_from_query(augmented_query)

# ✅ 2) 해당 도시 전용 리트리버 생성
retriever = city_retriever(city)

# ✅ 3) 검색
docs = retriever.invoke(augmented_query)

# ✅ 4) 출력
if not docs:
    print("⚠️ 검색 결과 없음 — metadata['city'] 존재 여부 확인 필요")
else:
    for d in docs:
        print("city:", d.metadata.get("city"))
        print("source:", d.metadata.get("source"))
        print("content:", d.page_content[:200], "...")
        print("--------------")


city: 뉴욕시
source: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap9\OneNYC_2050_Strategic_Plan.pdf
content: Source: Gregg Richards. ...
--------------
city: 뉴욕시
source: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap9\OneNYC_2050_Strategic_Plan.pdf
content: Source: Rae Breaux ...
--------------
city: 뉴욕시
source: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap9\OneNYC_2050_Strategic_Plan.pdf
content: WORK TO BE COMPLETED ...
--------------
city: 뉴욕시
source: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap9\OneNYC_2050_Strategic_Plan.pdf
content: SEPTEMBER 2018 LISTEN
JANUARY 2019 TEST
WHO WE HEARD FROM ...
--------------
city: 뉴욕시
source: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap9\OneNYC_2050_Strategic_Plan.pdf
content: INITIATIVE 3 OF 30 ...
--------------
city: 뉴욕시
source: C:\Users\ziclp\OneDrive\Desktop\LLM Assignment\1week\PRACTICE1\chap9\OneNYC_2050_Strategic_Plan.pdf
content: So

In [24]:
from langchain_core.messages import HumanMessage, AIMessage

# chat_history가 list라면
chat_history.append(HumanMessage(content=query_for_nyc))

# invoke
answer_msg = document_chain.invoke({
    "messages": chat_history,
    "query": augmented_query or query_for_nyc,
})

# LLM 답변도 content로 append
chat_history.append(AIMessage(content=answer_msg.content))

print(answer_msg.content)


제공된 문서에는 뉴욕시의 주거 정책에 대한 정보가 없습니다.


In [25]:
from langchain_core.messages import HumanMessage, AIMessage

# 0) q 확정 (증강 결과가 있으면 그걸 쓰고, 없으면 원질의)
q = (augmented_query or query_for_nyc or "").strip()
if not q:
    raise ValueError("질의가 비어 있습니다. query_for_nyc/augmented_query를 확인하세요.")

# 1) chat_history 정리: 비어있는 메시지 제거
def _nonempty(m): 
    return getattr(m, "content", None) and str(m.content).strip()
chat_history = [m for m in (chat_history or []) if _nonempty(m)]

# 2) 사용자 메시지 기록
chat_history.append(HumanMessage(content=q))

# 3) 도시 감지 → 도시 필터 리트리버
city = detect_city_from_query(q)
retriever = city_retriever(city)

# 4) 검색 (필터) → 없으면 무필터 폴백으로 최소 1~3개 확보
docs = retriever.invoke(q)
if not docs:
    # 폴백: 필터 제거해 최소한 컨텍스트 확보 (인덱스/메타 점검용)
    docs = vectorstore.as_retriever(search_kwargs={'k': 3}).invoke(q)

# 5) document_chain 호출에 필요한 payload 구성
payload = {"messages": chat_history, "context": docs}
try:
    props = getattr(document_chain, "input_schema").schema().get("properties", {})
except Exception:
    props = {}
if "city" in props:  payload["city"]  = city
if "query" in props: payload["query"] = q

# 6) 체인 호출 & 기록/출력
answer_msg = document_chain.invoke(payload)
answer_text = getattr(answer_msg, "content", str(answer_msg))
chat_history.append(AIMessage(content=answer_text))
print(answer_text)

# 7) (권장) 빠른 진단 출력
print(f"[DEBUG] city={city}, docs={len(docs)}")
if docs:
    print("first doc meta:", docs[0].metadata)


c:\Users\ziclp\AppData\Local\Programs\Python\Python313\Lib\site-packages\langchain_google_genai\chat_models.py:2348: UserWarning: HumanMessage with empty content was removed to prevent API error
  warnings.warn(


제공된 문서에는 뉴욕시의 주거 정책에 대한 정보가 없습니다. 서울시 관련 내용만 답변할 수 있습니다.
[DEBUG] city=뉴욕시, docs=10
first doc meta: {'moddate': '2020-01-03T15:55:12-05:00', 'title': '', 'producer': 'Adobe PDF Library 15.0', 'creationdate': '2019-04-30T13:48:30-04:00', 'page': 319, 'total_pages': 332, 'creator': 'Adobe InDesign 14.0 (Windows)', 'trapped': '/False', 'page_label': '320', 'source': 'C:\\Users\\ziclp\\OneDrive\\Desktop\\LLM Assignment\\1week\\PRACTICE1\\chap9\\OneNYC_2050_Strategic_Plan.pdf', 'city': '뉴욕시'}
